# Canadian Banks Stock Web Scraping from Yahoo Finance

## 1. Introduction

The purpose of this project is to analyze the stock performance of Canada’s leading banks — **BMO, CIBC, TD, RBC, and Scotiabank** — over the past five years. Through this analysis, we aim to provide data-driven insights into market trends, investment risks, and stock relationships. The findings can support investors, analysts, and financial professionals in making informed decisions.

### Objectives:
- **[Stock Market Performance Analysis](#4-exploratory-data-analysis-eda)** – Evaluating stock trends and overall performance.
- **[Volatility Assessment](#5-statistical-analysis--financial-metrics)** – Measuring price fluctuations to identify risk levels and market stability.
- **[Stock Price Correlation](#5-statistical-analysis--financial-metrics)** – Examining relationships between different banks' stock prices.
- **[Prediction Model Development](#5-statistical-analysis--financial-metrics)** – Building a predictive model to forecast stock trends and help investors determine which bank is more stable or offers better returns.

We collect stock price data and financial statements from [Yahoo Finance](https://finance.yahoo.com/), process them using Python, store them in a PostgreSQL database, and visualize insights using Power BI.

## 2. Data Collection

### 2.1 Install and Import Libraries

In [1]:
%%capture
!pip install yfinance
!pip install yahooquery
%pip install matplotlib
!pip install seaborn

In [2]:
import yfinance as yf
from yahooquery import Ticker
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [3]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### 2.2 Scrape Data for Selected Banks

#### Scrape Historical stock prices

In [4]:
# Step 1: Define the tickers for Canadian banks
bank_tickers = {
    "TD": "TD.TO",
    "BMO": "BMO.TO",
    "RBC": "RY.TO",
    "CIBC": "CM.TO",
    "Scotiabank": "BNS.TO"
}

In [35]:
# Step 2: Download historical stock prices
historical_data = {}
for bank, ticker in bank_tickers.items():
    print(f"Downloading historical data for {bank} ({ticker})...")
    historical_data[bank] = yf.download(ticker, start="2020-01-01", end="2025-01-01")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

In [36]:
# Step 3: Save historical data to CSV
for bank, data in historical_data.items():
    data.to_csv(f"{bank}_historical_data.csv")
    print(f"Saved historical data for {bank} to {bank}_historical_data.csv")

Saved historical data for TD to TD_historical_data.csv
Saved historical data for BMO to BMO_historical_data.csv
Saved historical data for RBC to RBC_historical_data.csv
Saved historical data for CIBC to CIBC_historical_data.csv
Saved historical data for Scotiabank to Scotiabank_historical_data.csv


In [37]:
# Step 4: Fetch financial data (e.g., market cap, PE ratio, dividend yield)
financial_data = {}
for bank, ticker in bank_tickers.items():
    print(f"Fetching financial data for {bank} ({ticker})...")
    stock = yf.Ticker(ticker)
    financial_data[bank] = {
        "Market Cap": stock.info.get("marketCap"),
        "PE Ratio": stock.info.get("trailingPE"),
        "Dividend Yield": stock.info.get("dividendYield"),
        "Beta": stock.info.get("beta"),
    }

Fetching financial data for TD (TD.TO)...
Fetching financial data for BMO (BMO.TO)...
Fetching financial data for RBC (RY.TO)...
Fetching financial data for CIBC (CM.TO)...
Fetching financial data for Scotiabank (BNS.TO)...


In [38]:
# Step 5: Save financial data to CSV
financial_df = pd.DataFrame(financial_data).transpose()
financial_df.to_csv("financial_data.csv")
print("Saved financial data to financial_data.csv")

Saved financial data to financial_data.csv


In [39]:
# Step 6: Preview the saved data
print("Sample historical data for TD:")
print(historical_data["TD"].head())

Sample historical data for TD:
Price           Close       High        Low       Open   Volume
Ticker          TD.TO      TD.TO      TD.TO      TD.TO    TD.TO
Date                                                           
2020-01-02  57.946808  57.994167  57.560077  57.781066  2207900
2020-01-03  57.899429  57.946782  57.544269  57.678440  3472900
2020-01-06  57.907352  57.986276  57.583757  57.615327  8359000
2020-01-07  57.844193  58.175676  57.788945  58.017829  3622600
2020-01-08  58.294071  58.562414  57.796845  57.867875  6465500


In [40]:
print("\nFinancial data:")
print(financial_df)


Financial data:
              Market Cap   PE Ratio  Dividend Yield   Beta
TD          1.435082e+11  17.372883          0.0512  0.822
BMO         1.018065e+11  14.665615          0.0462  1.160
RBC         2.403518e+11  15.391304          0.0323  0.842
CIBC        8.322114e+10  12.130494          0.0402  1.128
Scotiabank  9.780933e+10  12.243612          0.0531  0.978


#### Scrape financial data for each bank

In [41]:
# Function to scrape financial data for each bank
def scrape_financial_data():
    for bank, ticker in bank_tickers.items():
        print(f"Scraping financial data for {bank} ({ticker})...")

        # Create Ticker object
        stock = Ticker(ticker)

        # Get financial data
        income_statement = stock.income_statement()
        balance_sheet = stock.balance_sheet()
        cash_flow = stock.cash_flow()

        # Fetch dividends using the history method
        dividends = stock.history(period="max")  # Get full historical data
        dividends = dividends[dividends.index.get_level_values("symbol") == ticker]  # Filter for current ticker
        dividends = dividends[["dividends"]]  # Select only dividends column

        # Convert to DataFrame and save as CSV
        pd.DataFrame(income_statement).to_csv(f"{bank}_income_statement.csv", index=False)
        pd.DataFrame(balance_sheet).to_csv(f"{bank}_balance_sheet.csv", index=False)
        pd.DataFrame(cash_flow).to_csv(f"{bank}_cash_flow.csv", index=False)
        
        if not dividends.empty:
            dividends.to_csv(f"{bank}_dividends.csv")

        print(f"Data saved for {bank} ✅")

In [42]:
# Run the scraper
scrape_financial_data()

Scraping financial data for TD (TD.TO)...
Data saved for TD ✅
Scraping financial data for BMO (BMO.TO)...
Data saved for BMO ✅
Scraping financial data for RBC (RY.TO)...
Data saved for RBC ✅
Scraping financial data for CIBC (CM.TO)...
Data saved for CIBC ✅
Scraping financial data for Scotiabank (BNS.TO)...
Data saved for Scotiabank ✅
